In [2]:
#Dependencies
import requests
import json
from api_keys import weather_api_key
import numpy as np
import pandas as pd
from citipy import citipy
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress

url = "http://api.openweathermap.org/data/2.5/weather?"

ModuleNotFoundError: No module named 'dotenv'